<a href="https://colab.research.google.com/github/JTEC-Iritani/ICT-Training-2021/blob/main/python/Practice_14_Kuching_Traffic_FBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kuching Traffic Data

In [ ]:
# Import the libraries 使用するライブラリのインポート
import pandas as pd
import numpy as np
import pip
import datetime as dt
from fbprophet import Prophet
import matplotlib.pyplot as plt #グラフ用
# import openlocationcode
# !pip openlocationcode
import folium #地図作成用
from folium.plugins import HeatMap # ヒートマップ用ライブラリ

In [ ]:
# ローカルファイル（クチン市交通量データ）をアップロードする（Google Colaboratoryの場合）
from google.colab import files
uploaded = files.upload()

In [ ]:
# pickleデータをDataFrameに変換（フォルダ名を必要に応じ修正すること）
df = pd.read_pickle('data/Kuching_Traffic_Data_202003_1.pickle')
# データの内容を表示
df
# df.head(3)

In [ ]:
df.info()
# df.describe()

In [ ]:
INT_List = df.INT_ID.unique()
INT_List

In [ ]:
df1 = df[df['INT_ID'] == 8207] #交差点IDを指定してデータを抽出
df1

In [ ]:
# df1.describe()
df1.info()

In [ ]:
# Sample 1日単位より短い間隔のデータ
# m = Prophet(changepoint_prior_scale=0.01).fit(df)
# future = m.make_future_dataframe(periods=300, freq='H')
fcst = m.predict(future)
fig = m.plot(fcst)

In [ ]:
#Python
# m = Prophet().fit(df1)
m = Prophet(changepoint_prior_scale=0.01).fit(df1)

In [ ]:
# Python
future = m.make_future_dataframe(periods=168, freq='H')
# future = m.make_future_dataframe(periods=365)

future.tail()

In [ ]:
# Python
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# Python
fig1 = m.plot(forecast)

In [ ]:
# Python
fig2 = m.plot_components(forecast)

In [ ]:
# Google Colabの場合：ローカルファイル（クチン市交差点位置情報データ）をアップロードする（ファイル名：Kuching Intersections GPS.csv）
# The Littlest JupyterHubの場合：下記の2行は不要。別途マニュアルでCSVファイルをアップロードしておく
from google.colab import files
uploaded = files.upload()

In [ ]:
# CSVをDataFrameに変換して表示（フォルダ名を必要に応じ修正すること）
df_kuching = pd.read_csv('data/Kuching Intersections GPS.csv')
df_kuching.head(8)

In [ ]:
# 地図オブジェクト作成
map_kuching = folium.Map(location=[1.523833, 110.347119], zoom_start=12)

# マーカープロット（測定交差点は青、それ以外は赤）
for i, row in df_kuching.iterrows():
    if (row['INT_ID'] in INT_List):
    # if (row['INT_ID'] == 8207) | (row['INT_ID'] == 8208) | (row['INT_ID'] == 8226):
        folium.Marker(
            location=[row['LAT'], row['LNG']],
            popup=row['INT_NAME'],
            icon=folium.Icon(color='blue')
        ).add_to(map_kuching)
    else:
        folium.Marker(
            location=[row['LAT'], row['LNG']],
            popup=row['INT_NAME'],
            icon=folium.Icon(color='red')
        ).add_to(map_kuching)
  
  # 地図表示
map_kuching